This notebook performs basic text preprocessing on the unified multilingual dataset to prepare it for feature extraction and modeling.

In [4]:
import pandas as pd
import re
import unicodedata
import pandas as pd
import re
import string

# Load unified dataset
df = pd.read_csv("../dataset/unified_dataset.csv")

print("Dataset Shape:", df.shape)
df.head()


Dataset Shape: (81964, 3)


,text,label,language
0,‘मोदी के शासन के दौरान गंगा’ गंगा नदी नरेन्द्...,1,hi
1,यह खबर आने से पहले छवि क्रेडिट जस्टिन सुलिवान/...,1,hi
2,गुलाब गेंद वाल डे-नाइट टेस्ट मैच कप्ता विराट क...,0,hi
3,उत्तर कोरिया रॉकेट प्रक्षेपण योजनाएं 71 0 15 0...,1,hi
4,राष्ट्रपति डोनाल्ड ट्रम्प और प्रथम महिला मेलान...,0,hi


In [5]:
def clean_text(text: str) -> str:
    """Language-agnostic text cleaner that preserves characters from all scripts."""
    if not isinstance(text, str):
        text = str(text)

    # Normalize unicode so similar characters share the same code point
    text = unicodedata.normalize("NFKC", text)

    # Remove URLs, mentions, and HTML entities
    text = re.sub(r"https?://\S+|www\.\S+", " ", text)
    text = re.sub(r"@[A-Za-z0-9_]+", " ", text)
    text = re.sub(r"&[a-z]+;", " ", text)

    # Lowercase for consistency across languages with case
    text = text.lower()

    # Keep letters, numbers, and combining marks (e.g., Hindi matras)
    allowed_categories = ("L", "N")
    mark_categories = {"Mn", "Mc", "Me"}
    cleaned_chars = []
    for char in text:
        cat = unicodedata.category(char)
        if char.isspace():
            cleaned_chars.append(" ")
        elif cat.startswith(allowed_categories) or cat in mark_categories:
            cleaned_chars.append(char)
        else:
            cleaned_chars.append(" ")

    text = ''.join(cleaned_chars)
    text = re.sub(r"\s+", " ", text).strip()
    return text


# Apply cleaner
df['clean_text'] = df['text'].apply(clean_text)

# Quick preview
df[['language', 'text', 'clean_text']].head()

,language,text,clean_text
0,hi,‘मोदी के शासन के दौरान गंगा’ गंगा नदी नरेन्द्...,मोदी के शासन के दौरान गंगा गंगा नदी नरेन्द्र म...
1,hi,यह खबर आने से पहले छवि क्रेडिट जस्टिन सुलिवान/...,यह खबर आने से पहले छवि क्रेडिट जस्टिन सुलिवान ...
2,hi,गुलाब गेंद वाल डे-नाइट टेस्ट मैच कप्ता विराट क...,गुलाब गेंद वाल डे नाइट टेस्ट मैच कप्ता विराट क...
3,hi,उत्तर कोरिया रॉकेट प्रक्षेपण योजनाएं 71 0 15 0...,उत्तर कोरिया रॉकेट प्रक्षेपण योजनाएं 71 0 15 0...
4,hi,राष्ट्रपति डोनाल्ड ट्रम्प और प्रथम महिला मेलान...,राष्ट्रपति डोनाल्ड ट्रम्प और प्रथम महिला मेलान...


In [6]:
# Keep only required columns
final_df = df[['clean_text', 'label', 'language']].copy()

# Save cleaned dataset
output_path = "../dataset/unified_cleaned_dataset.csv"
final_df.to_csv(output_path, index=False, encoding="utf-8")

print("✅ Cleaned dataset saved successfully!")
print("Shape:", final_df.shape)
print("Saved to:", output_path)

✅ Cleaned dataset saved successfully!
Shape: (81964, 3)
Saved to: ../dataset/unified_cleaned_dataset.csv
